In [7]:
import pandas as pd
import os
import logging

# Read the CSV dataset
file_path = '../datasets/movies_all.csv'
df = pd.read_csv(file_path)

df = df.fillna('')

print(len(df))
df.head()

10638


,id,title,summary,year,genre,certificate,runtime,rating,votes,sentiment,img
0,15239678,Dune: Part Two,Paul Atreides unites with Chani and the Fremen while seeking revenge against the conspirators who destroyed his family.,2024,Adventure,PG-13,2h 46m,8.7,360000,-0.7650,https://m.media-amazon.com/images/M/MV5BN2QyZGU4ZDctOWMzMy00NTc5LThlOGQtODhmNDI1NmY5YzAwXkEyXkFqcGdeQXVyMDM2NDM2MQ@@.jpg
1,14539740,Godzilla x Kong: The New Empire,"Two ancient titans, Godzilla and Kong, clash in an epic battle as humans unravel their intertwined origins and connection to Skull Island's mysteries.",2024,Adventure,PG-13,1h 55m,6.5,37000,-0.3818,https://m.media-amazon.com/images/M/MV5BY2QwOGE2NGQtMWQwNi00M2IzLThlNWItYWMzNGQ5YWNiZDA4XkEyXkFqcGdeQXVyNTE1NjY5Mg@@.jpg
2,23137904,Rebel Moon - Part Two: The Scargiver,"Kora and surviving warriors prepare to defend Veldt, their new home, alongside its people against the Realm. The warriors face their pasts, revealing their motivations before the Realm's forces arrive to crush the growing rebellion.",2024,Adventure,PG-13,2h 2m,5.2,17000,0.2023,"https://m.media-amazon.com/images/M/MV5BYmQ2ODgyZjMtMmQyMy00ZWRhLTllOGUtZDI3MDkzNjFhZWQ2XkEyXkFqcGdeQXVyMTUzMTg2ODkz._V1_QL75_UX140_CR0,0,140,207_.jpg@.jpg"
3,21692408,Kung Fu Panda 4,"After Po is tapped to become the Spiritual Leader of the Valley of Peace, he needs to find and train a new Dragon Warrior, while a wicked sorceress plans to re-summon all the master villains whom Po has vanquished to the spirit realm.",2024,Adventure,PG,1h 34m,6.4,30000,-0.8957,"https://m.media-amazon.com/images/M/MV5BZDY0YzI0OTctYjVhYy00MTVhLWE0NTgtYTRmYTBmOTE3YTViXkEyXkFqcGdeQXVyMTUzMTg2ODkz._V1_QL75_UX140_CR0,7,140,207_.jpg@.jpg"
4,1160419,Dune,A noble family becomes embroiled in a war for control over the galaxy's most valuable asset while its heir becomes troubled by visions of a dark future.,2021,Adventure,PG-13,2h 35m,8.0,851000,0.4890,https://m.media-amazon.com/images/M/MV5BMDQ0NjgyN2YtNWViNS00YjA3LTkxNDktYzFkZTExZGMxZDkxXkEyXkFqcGdeQXVyODE5NzE3OTE@.jpg


In [8]:
# Create a new column named 'metadata' with all the data combined
def generate_metadata(row):
    metadata = (
        f"{row['title']} | "
        f"{row['summary']}"
    )
    return metadata

df['metadata'] = df.apply(generate_metadata, axis=1)

pd.set_option('display.max_colwidth', None)
print("A sample value from column 'metadata':")
df['metadata'].head(1)

A sample value from column 'metadata':


0    Dune: Part Two | Paul Atreides unites with Chani and the Fremen while seeking revenge against the conspirators who destroyed his family.
Name: metadata, dtype: object

In [10]:
from openai import OpenAI
import openai
from tqdm import tqdm

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

client = OpenAI(
    api_key=os.getenv('OPENAI_API_KEY'),
)

# Generate the vector embeddings for selected columns
def get_embedding(text, model='text-embedding-3-small'):
    try:
        # text = text.replace('\n', ' ')
        return client.embeddings.create(input = [text], model=model).data[0].embedding
    except Exception as e:
        logging.error(f'Error generating embeddings: {e}. Found issue in the following text: {text}.')
        text = 'No data available'
        return client.embeddings.create(input = [text], model=model).data[0].embedding

tqdm.pandas()
df['metadata_vector'] = df['metadata'].progress_apply(lambda x: get_embedding(x, model='text-embedding-3-small'))
print('Conversion to vector embeddings has been completed.')

df.head()

100%|███| 10638/10638 [30:17<00:00,  5.85it/s]

Conversion to vector embeddings has been completed.


,id,title,summary,year,genre,certificate,runtime,rating,votes,sentiment,img,metadata,metadata_vector
0,15239678,Dune: Part Two,Paul Atreides unites with Chani and the Fremen while seeking revenge against the conspirators who destroyed his family.,2024,Adventure,PG-13,2h 46m,8.7,360000,-0.7650,https://m.media-amazon.com/images/M/MV5BN2QyZGU4ZDctOWMzMy00NTc5LThlOGQtODhmNDI1NmY5YzAwXkEyXkFqcGdeQXVyMDM2NDM2MQ@@.jpg,Dune: Part Two | Paul Atreides unites with Chani and the Fremen while seeking revenge against the conspirators who destroyed his family.,"[-0.011729336343705654, 0.04041394963860512, 0.016686340793967247, -0.02490115351974964, -0.052027154713869095, 0.009987355209887028, 0.014791554771363735, 0.0568191297352314, -0.0030515221878886223, 0.013092359527945518, -0.006546179763972759, -0.023055266588926315, -0.006766219157725573, 0.010182945989072323, -0.001870337058790028, 0.02125827595591545, 0.023886526003479958, -0.02572019025683403, -0.0015693105524405837, 0.06513173878192902, -0.04046284779906273, 0.009590061381459236, 0.00999957975000143, 0.0031966872047632933, 0.011270919814705849, 0.006151942070573568, -0.009290562942624092, 0.02559794671833515, 0.04767525941133499, -0.009492265991866589, 0.03655103221535683, -0.02221178077161312, 0.055107709020376205, 0.006876239087432623, -0.02263963595032692, -0.00860599521547556, -0.046428367495536804, 0.01305568590760231, -0.022334024310112, 0.0018963139737024903, -0.013422418385744095, -0.02958310954272747, -0.016160689294338226, 0.016185138374567032, 0.03349492326378822, -0.04202757403254509, -0.010036253370344639, 0.00850208755582571, 0.012970115058124065, -0.004483307711780071, 0.004373288247734308, -0.011032544076442719, 0.007774734403938055, 0.06190448999404907, -0.037235599011182785, 0.005143426824361086, -0.06077984347939491, 0.007463011424988508, 0.004779750015586615, -0.030243227258324623, -0.005060912109911442, 0.0002519377740100026, 0.02243182063102722, 0.012377230450510979, 0.005018126685172319, -0.03662437945604324, -0.008159804157912731, -0.002735215239226818, -0.005342073738574982, 0.034937407821416855, -0.00982232578098774, -7.396731962217018e-05, -0.02142941765487194, -0.026698144152760506, 0.03630654141306877, -0.02650255337357521, 0.047381870448589325, -0.014742656610906124, -0.049068842083215714, 0.0038995917420834303, -0.03545083478093147, -0.029240824282169342, -0.049460023641586304, -0.029949842020869255, 0.011894365772604942, -0.03699111193418503, -0.021845048293471336, 0.0031936310697346926, -0.08703790605068207, -0.021197153255343437, 0.014803779311478138, 0.02325085736811161, -0.010653586126863956, -0.004590271506458521, -2.1679254132322967e-05, 0.013373521156609058, 0.009449481032788754, -0.0058310506865382195, -0.026209168136119843, -0.014901574701070786, ...]"
1,14539740,Godzilla x Kong: The New Empire,"Two ancient titans, Godzilla and Kong, clash in an epic battle as humans unravel their intertwined origins and connection to Skull Island's mysteries.",2024,Adventure,PG-13,1h 55m,6.5,37000,-0.3818,https://m.media-amazon.com/images/M/MV5BY2QwOGE2NGQtMWQwNi00M2IzLThlNWItYWMzNGQ5YWNiZDA4XkEyXkFqcGdeQXVyNTE1NjY5Mg@@.jpg,"Godzilla x Kong: The New Empire | Two ancient titans, Godzilla and Kong, clash in an epic battle as humans unravel their intertwined origins and connection to Skull Island's mysteries.","[0.007644626311957836, 0.03877980634570122, 0.010047695599496365, 0.010229579173028469, -0.02345748245716095, 0.02733766846358776, -0.011761811561882496, 0.03595785051584244, -0.030909202992916107, 0.012985392473638058, 0.014705020003020763, -0.04594491794705391, -0.025948738679289818, -0.034568920731544495, -0.025441668927669525, -0.0035853132139891386, -0.07015198469161987, -0.05710045248270035, 0.009887858293950558, 0.01654590293765068, -0.031041480600833893, 0.00035791125264950097, 0.0336209237575531, 0.01655692607164383, -0.007170626427978277, 0.041557665914297104, -0.010273671709001064, 0.08271849155426025, 0.06591905653476715, 0.021252831444144

In [11]:
df.to_csv('movies_vector.csv', index=False)